1. train되어 있는 model에 s_j 기준 내림차순 정렬
2. 하위 channel 제거
3. 40 epoch retraining
4. Test Accuracy, #FLOPs

In [27]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.nn as nn
import tensorflow as tf

import torch.optim as optim
from torch.optim import lr_scheduler

import torch.nn.init as init
import kornia
import math

import sys
sys.path.append('/home/hslee/Desktop/Embedded_AI/PyTorch_Tutorials/03_Pruning_Filters_for_Efficient_Convnets/')
from architecture import *
from utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 1. train되어 있는 model에 s_j 기준 내림차순 정렬

In [28]:
model = VGG16_BN()
checkpoint = torch.load('../00_vgg16_baseline_exp1/checkpoint/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [29]:
layer_idx = 1
for layer in model.modules():
    if isinstance(layer, nn.Conv2d):
        print(f"conv {layer_idx} : {layer.weight.data.shape}")
        layer_idx += 1
    if isinstance(layer, nn.Linear):
        print(f"fc : {layer.weight.data.shape}")
        layer_idx += 1

conv 1 : torch.Size([64, 3, 3, 3])
conv 2 : torch.Size([64, 64, 3, 3])
conv 3 : torch.Size([128, 64, 3, 3])
conv 4 : torch.Size([128, 128, 3, 3])
conv 5 : torch.Size([256, 128, 3, 3])
conv 6 : torch.Size([256, 256, 3, 3])
conv 7 : torch.Size([256, 256, 3, 3])
conv 8 : torch.Size([512, 256, 3, 3])
conv 9 : torch.Size([512, 512, 3, 3])
conv 10 : torch.Size([512, 512, 3, 3])
conv 11 : torch.Size([512, 512, 3, 3])
conv 12 : torch.Size([512, 512, 3, 3])
conv 13 : torch.Size([512, 512, 3, 3])
fc : torch.Size([512, 512])
fc : torch.Size([10, 512])


In [46]:
import copy
layer = 0
fc_layer = 0
filters_pruned_away_list = [50, 50, 0, 0, 0, 0, 0, 50, 75, 75, 75, 75, 75, 50, 0]
for param in model.modules() :
    # if conv layer
    if isinstance(param, torch.nn.Conv2d) :
        print(f"="*40,f" conv{layer+1} ", "="*40)
        weight = param.weight.data
        pruned_rate = filters_pruned_away_list[layer]
        num_prune_channels = round(weight.shape[0] * pruned_rate / 100)
        print(f"pruned_rate : {pruned_rate}%")
        print(f"num_prune_channels : {num_prune_channels}")
        
        model = getPrunedNetwork(model, layer, num_prune_channels)
        layer += 1
        
    # if fc layer
    elif isinstance(param, torch.nn.Linear) :
        print(f"="*40,f" fc {(layer % 12)}", "="*40)
        weight = param.weight.data
        pruned_rate = filters_pruned_away_list[layer]
        num_prune_channels = round(weight.shape[0] * pruned_rate / 100)
        print(f"pruned_rate : {pruned_rate}%")
        print(f"num_prune_channels : {num_prune_channels}")
        
        model = getPrunedNetwork(model, layer, num_prune_channels, "fc")
        layer += 1

========================================  conv1  ========================================
pruned_rate : 50%
num_prune_channels : 0
weight.shape : torch.Size([0, 3, 3, 3])
bias.shape : torch.Size([0])
bn_gamma.shape : torch.Size([0])
bn_beta.shape : torch.Size([0])
bn_running_mean.shape : torch.Size([0])
bn_running_var.shape : torch.Size([0])
sorted_weight_indices : tensor([], dtype=torch.int64)
saving_filter_idices : tensor([], dtype=torch.int64)
pruned_weight.shape : torch.Size([0, 3, 3, 3])
pruned_bias.shape : torch.Size([0])
pruned_bn_gamma.shape : torch.Size([0])
pruned_bn_beta.shape : torch.Size([0])
pruned_bn_running_mean.shape : torch.Size([0])
pruned_bn_running_var.shape : torch.Size([0])
pruned_next_weight.shape : torch.Size([0, 0, 3, 3])
========================================  conv2  ========================================
pruned_rate : 50%
num_prune_channels : 0
weight.shape : torch.Size([0, 0, 3, 3])
bias.shape : torch.Size([0])
bn_gamma.shape : torch.Size([0])
bn_beta.s

TypeError: getPrunedNetwork() takes 3 positional arguments but 4 were given

# 2. pruning smallest channels

# 3. 40 epoch retraining

# 4. Test Accuracy, #FLOPs